In [1]:
from appgeopy import *
from my_packages import *

# 2025/10/24

After successfully model the `diff. disp.` relationship between MLCW and InSAR using GWR,
we try to test if we can predict the future `diff. disp.` based on the timeseries of $\beta_{1}$ and $\beta_{0}$

Work flow:

1. Values of `diff. disp.` from MLCW in 2023 - 2024 to validate (get it from `GTWR` input files)
2. Values of `diff. disp.` from InSAR in 2023 - 2024 to calculate the predicted values (get it from `GTWR` input files as well)
3. Filter the stations with available MLCW values in 2023 - 2024
4. `Approach 1`: Get the average and stdev of coefficient & intercept --> perform prediction
5. `Approach 2`: Model the signal of coefficient & intercept with SARIMAX --> perform prediction


#### Preparation

Prepare the diff. disp. value for 2023 - 2024 period

#### Approach 1

Get the average and stdev of coefficient & intercept --> perform prediction

#### Approach 2

Apply SARIMAX to forecast the values of coefficient & intercept in period 2023 - 2024 --> perform prediction

In [3]:
from utils2 import *

files = glob(os.path.join("reg_diffdisp/", "*csv"))
results_csv = glob(os.path.join("GWR*Output*", "*combined_results.csv"))
# idx = 0

n_seasonal = 1

for idx in trange(len(files)):

    select_file = files[idx]
    select_result = results_csv[idx]

    df = pd.read_csv(select_file)
    result_df = pd.read_csv(select_result).rename(
        {"input_STATION": "STATION"}, axis=1
    )

    current_layer = "_".join(
        os.path.basename(select_file).split(".")[0].split("_")[-2:]
    )
    output_dir = f"arima_output_seasonal{n_seasonal}/{current_layer}"

    # Process all stations
    all_metrics = []
    all_mlcw_metrics = []

    for station in df["STATION"].unique():
        try:
            df_station = df.query("STATION==@station")
            result_station = result_df.query("STATION==@station")
            future_dates = pd.to_datetime(df_station["time"])

            # Fit ARIMA and save predictions
            metrics = process_single_station_with_predictions(
                df_station,
                result_station,
                station,
                current_layer,
                future_dates,
                output_dir,
                n_seasonal=n_seasonal,
            )
            all_metrics.append(metrics)

            # Generate MLCW predictions
            insar_data = df_station[["time", "DIFFDISP"]].copy()
            insar_data["time"] = pd.to_datetime(insar_data["time"])

            mlcw_predictions = predict_mlcw_from_arima(
                station_name=station,
                insar_data=insar_data,
                predictions_dir=output_dir,
                layer_name=current_layer,
            )

            # Save MLCW predictions
            mlcw_predictions.to_csv(
                f"{output_dir}/{station}_MLCW_predictions.csv", index=False
            )

            # Visualize and evaluate MLCW predictions
            mlcw_metrics = plot_mlcw_prediction_vs_observed(
                mlcw_predictions=mlcw_predictions,
                observed_data=df_station[["time", current_layer]],
                station_name=station,
                layer_name=current_layer,
                output_dir=output_dir,
            )
            all_mlcw_metrics.append(mlcw_metrics)

            print(f"✓ {station}")
        except Exception as e:
            print(f"✗ {station}: {e}")

    # Save all metrics
    pd.DataFrame(all_metrics).to_excel(
        f"{output_dir}/arima_fitting_metrics.xlsx", index=False
    )
    pd.DataFrame(all_mlcw_metrics).to_excel(
        f"{output_dir}/mlcw_prediction_metrics.xlsx", index=False
    )

    print(f"\nProcessing complete. Results saved to {output_dir}/")

  0%|          | 0/5 [00:00<?, ?it/s]

✓ BEICHEN
✓ GUANGFU
✓ HONGLUN
✓ HUNAN
✓ HUWEI
✓ JIAXING
✓ KECUO
✓ NEILIAO
✓ QIAOYI
✓ TUKU
✓ XINSHENG
✓ XIUTAN
✓ XIZHOU
✓ YUANCHANG

Processing complete. Results saved to arima_output_seasonal1/Layer_1/
✓ BEICHEN
✓ GUANGFU
✓ HONGLUN
✓ HUNAN
✓ HUWEI
✓ JIAXING
✓ KECUO
✓ NEILIAO
✓ QIAOYI
✓ TUKU
✓ XINSHENG
✓ XIUTAN
✓ XIZHOU
✓ YUANCHANG

Processing complete. Results saved to arima_output_seasonal1/Layer_2/
✓ BEICHEN
✓ GUANGFU
✓ HONGLUN
✓ HUNAN
✓ HUWEI
✓ JIAXING
✓ KECUO
✓ NEILIAO
✓ QIAOYI
✓ TUKU
✓ XINSHENG
✓ XIUTAN
✓ XIZHOU
✓ YUANCHANG

Processing complete. Results saved to arima_output_seasonal1/Layer_3/
✓ BEICHEN
✓ GUANGFU
✓ HONGLUN
✓ HUNAN
✓ HUWEI
✓ KECUO
✓ NEILIAO
✓ QIAOYI
✓ TUKU
✓ XINSHENG
✓ XIUTAN
✓ XIZHOU
✓ YUANCHANG

Processing complete. Results saved to arima_output_seasonal1/Layer_4/
✓ BEICHEN
✓ GUANGFU
✓ HONGLUN
✓ HUNAN
✓ HUWEI
✓ JIAXING
✓ KECUO
✓ NEILIAO
✓ QIAOYI
✓ TUKU
✓ XINSHENG
✓ XIUTAN
✓ XIZHOU
✓ YUANCHANG

Processing complete. Results saved to arima_output_seasonal1/Layer_All